In [1]:
import sys, os
import numpy as np
from PIL import Image, ImageDraw

In [ ]:
# PerlinNoise: An implementation typically involves three steps: defining a grid of random gradient vectors, computing the
# dot product between the gradient vectors and their offsets, and interpolation between these values.

# Define an n-dimensional grid where each grid intersection has associated with it a fixed random n-dimensional unit-length
# gradient vector, except in the one dimensional case where the gradients are random scalars between -1 and 1.

# For working out the value of any candidate point, first find the unique grid cell in which the point lies.
# Then the 2^n corners of that cell, and their associated gradient vectors, are identified.
# Next, for each corner, an offset vector is calculated, being the displacement vector from the candidate point to that corner.

# For each corner, we take the dot product between its gradient vector and the offset vector to the candidate point.
# This dot product will be zero if the candidate point is exactly at the grid corner.

In [51]:
class PerlinNoise(object):
    def __init__(self):
        self.d_theta=0.05

    def main(self, width=500, height=500, z=0, smoothing_function='', num_octaves=1, persistence=1, 
             grid_size=(100,100), layered_effect=False, contour=False):
        self.width = width
        self.height = height
        self.f = smoothing_function
        self.grid_size = grid_size
        
        np.random.seed(123)
        
        self.grads = np.random.uniform(-1, 1, (width, height, 2))
        
        norm = np.sqrt(np.sum(self.grads**2, 2))
        
        self.grads[:,:,0] /= norm
        self.grads[:,:,1] /= norm
        
        cos_t = np.cos(self.d_theta)
        sin_t = np.sin(self.d_theta)
        
        if self.d_theta:
            d0 = cos_t*self.grads[:,:,0] - sin_t*self.grads[:,:,1]
            d1 = sin_t*self.grads[:,:,0] + cos_t*self.grads[:,:,1]
            
            self.grads[:,:,0] = d0
            self.grads[:,:,1] = d1

        path='perlin_%s_z%d_oct%d_pers%.1f_grid%dx%d.png' % (smoothing_function, z, num_octaves, persistence,
                                                               self.grid_size[0], self.grid_size[1])
        print(path)

        im = Image.new("L", (width, height))

        m=np.zeros((width, height))
        for y in range(height):
            for x in range(width):
                pt = self.perlin_octaves(x, y, z, num_octaves, persistence)
                m[y, x]=pt
                
        if layered_effect:
            m = ((m + abs(m.min())) * 255)/m.max()
        else:
            m += abs(m.min())
            m = (m * 255)/m.max()

        if contour:
            m = np.array(m, dtype=int)
            m = np.multiply(m, np.array((125<=m), dtype=int) - np.array((m < 150), dtype=int))
            m = 255 - m

        im = Image.fromarray(m.astype('uint8'),'L')
        im.save(path)
            
        
    def perlin_octaves(self, x, y, z, num_octaves, pers):
        total=0
        freq=1
        amp=1
        max_val=0
        for i in range(num_octaves):
            total+=self.perlin(x*freq, y*freq, z*freq) * amp
            max_val += amp
            amp *= pers
            freq *=2
        return total/max_val

    
    def perlin(self, x, y, z):
        #id grid this point is in
        px1=int(x/self.grid_size[0])
        px2=px1+1
        py1=int(y/self.grid_size[1])
        py2=py1+1

        #range 0-1
        xingrid=(x%self.grid_size[0])/self.grid_size[0]
        yingrid=(y%self.grid_size[1])/self.grid_size[1]

        #distance vectors
        distx1y1=np.array([xingrid,yingrid])    #0,0
        distx1y2=np.array([xingrid,yingrid-1])  #0,1
        distx2y1=np.array([xingrid-1,yingrid])  #1,0
        distx2y2=np.array([xingrid-1,yingrid-1])#1,1

        gradx1y1=self.grads[px1,py1]
        gradx1y2=self.grads[px1,py2]
        gradx2y1=self.grads[px2,py1]
        gradx2y2=self.grads[px2,py2]
        
        x1y1dot = np.dot(gradx1y1,distx1y1)
        x1y2dot = np.dot(gradx1y2,distx1y2)
        x2y1dot = np.dot(gradx2y1,distx2y1)
        x2y2dot = np.dot(gradx2y2,distx2y2)
        
        #smoothed coords rel to grid
        if self.f=='smoothstep':
            cx=self.smoothstep(xingrid)
            cy=self.smoothstep(yingrid)
        elif self.f=='sigmoid':
            cx=self.sig(xingrid)
            cy=self.sig(yingrid)
        elif self.f=='tanh':
            cx=np.tanh(xingrid)
            cy=np.tanh(yingrid)
        else:
            cx=xingrid
            cy=yingrid
        
        #lin interpolate using smoothed coords
        xy1interp = self.linear_interp(x1y1dot,x2y1dot,cx)
        xy2interp = self.linear_interp(x1y2dot,x2y2dot,cx)
        interp = self.linear_interp(xy1interp,xy2interp,cy)
            
        return interp
    
    
    def smoothstep(self,t): #6t^5-15t^4+10t^3
        return t*t*t*(t*(t*6-15)+10)
    
    
    def sig(self,t):
        return 1/(1+np.exp(-t))
    
    
    def linear_interp(self,x0,x1,w):
        return x0 + w * (x1 - x0)

In [56]:
n = PerlinNoise()

#octaves - details, pers is contrast
n.main(width=100, height=100, z=0, smoothing_function='smoothstep',
       num_octaves=4, persistence=0.7, grid_size=(100,100), 
       layered_effect=True, contour=False)

perlin_smoothstep_z0_oct4_pers0.7_grid100x100.png
